# Lesson 3 Pick and Place

Welcome to the first challenge. Everything before was just messing around with the setup and basics for Lisp and CRAM. You've learned how to write Lisp code, define functions and parameters, create and visualize poses, and how to use one of the key components of CRAM: Designators. Now we will utilize all of that by making the robot pick and place objects from one place to another.

To use CRAM in this lesson we need to clean up a bit. Close the last Bullet World window. It is bound to the kernel of lesson 2, so it can't communicate with it when we work here in lesson 3. Then load the package, initialize the simulator again and define the tutorial package for your own code.

In [ ]:
(asdf:load-system :cram-pr2-pick-place-demo)

In [ ]:
(roslisp-utilities:startup-ros)

In [ ]:
(defpackage cram-bullet-world-tutorial
  (:nicknames :btw-tut)
  (:use :common-lisp :cram-prolog :desig :exe))

In [ ]:
(in-package :btw-tut)

We also want to re-use the `make-pose` function from lesson 2 again, so compile it here.

In [ ]:
(defun make-pose (frame point euler)
    (declare (type string frame)
             (type list point euler))
    "Creates a cl-transforms pose in 3D space w.r.t. the given frame, point and euler."
    (if (and (eq (length point) 3) (eq (length euler) 3))
        (destructuring-bind (ax ay az) (mapcar #'eval euler)
          (cl-transforms-stamped:make-pose-stamped frame 0.0 
                                                   (apply #'cl-transforms:make-3d-vector point)
                                                   (cl-transforms:euler->quaternion :ax ax :ay ay :az az)))
        (format T "point ~a or euler ~a is not of length 3" point euler)))

## Spawning Objects and simulating

We've spawned objects before, in lesson 2. This is how we did it.

In [ ]:
(btr-utils:spawn-object 'bottle-1 :bottle :pose (make-pose "map" '(-2.323 -1 0.82) '(0 0 0)))

Notice that the `spawn-object` function has more optional slots to fill.

In [ ]:
(describe 'btr-utils:spawn-object)

There are the argments name, type, pose, color, mass and world. Only the first two are mandadory, the other ones use default values if they are not set explicitly. We recommend to set the `pose` argument though, otherwise it will spawn in the coordinate (0 0 0). Mass doesn't really have any effect in bullet, unless it's 0, then it won't fall down.

You can set the color though.

In [ ]:
(btr-utils:spawn-object 'bottle-1 :bottle :pose (make-pose "map" '(-2.323 -1 1.82) '(0 0 0)) :color '(1 0 0))

Kill an object like this

In [ ]:
(btr-utils:kill-object 'bottle-1)
;; or (btr-utils:kill-all-objects) if you want to get rid of them all

To find all possible `obj-type`, the parameter `btr::*mesh-files*` contains the mesh files as key-value pairs between keyword and the location of their mesh-file. We can map over that list and return their keys. 

In [ ]:
(mapcar #'first btr::*mesh-files*)

After you've spawned an object, you can simulate the world to let it fall. Spawn your object higher above (increase the z-coordinate) and let it fall.

In [ ]:
(btr:simulate btr:*current-bullet-world* 10) ;; in seconds

If you want, you can do this in real time by simulating step by step. 

In [ ]:
(dotimes (i 10)
    (sleep 0.1)
    (btr:simulate btr:*current-bullet-world* 0.1))

And to make it look fancy, define a function for that.

In [ ]:
(defun simulate (&key (duration 5.0)    ;; in seconds 
                      (framerate 20.0)) ;; in Hz
    (let ((sleep-interval (/ 1.0 framerate))
          (times (floor (* duration framerate))))
         (dotimes (_ times)
             (sleep (/ sleep-interval 2))
             (btr:simulate btr:*current-bullet-world* (/ sleep-interval 2)))))

In [ ]:
(simulate)

In [ ]:
(simulate :duration 2)

In [ ]:
(simulate :framerate 5)

In [ ]:
(simulate :duration 1 :framerate 10)

I know what you're thinking. Here's how to make it rain.

In [ ]:
(let ((number-of-objects 20)
      (x-y-scatter-factor 0.8)
      (z-scatter-factor 0.5)
      (z-offset 2.0))
     (loop for n in (alexandria:iota number-of-objects)
           for object-name = (make-symbol (concatenate 'string "OBJECT-" (format NIL "~a" n)))
           for object-type = (nth (random (length btr::*mesh-files*)) (mapcar #'first btr::*mesh-files*))
           for x = (* (- (random 1.0) 0.5) x-y-scatter-factor)
           for y = (* (- (random 1.0) 0.5) x-y-scatter-factor)
           for z = (+ (* (random 1.0) z-scatter-factor) z-offset)
           do (btr-utils:spawn-object object-name object-type :pose `((,x ,y ,z) (0 0 0 1)))))

In [ ]:
(simulate)

In [ ]:
(btr-utils:kill-all-objects)

In [ ]:
(roslisp-utilities:startup-ros) ;; you're welcome

## Moving Around

Moving the robot is done by performing an action of type `going`. You can move the robot off the ground, but that seems unreasonable. The `going` action is very forgiving in that regard. Feel free to move the robot around. It will throw a `low-level-failure` if your taget pose would make the robot collide with the environment.

In [ ]:
(urdf-proj:with-simulated-robot
  (let ((?navigation-goal (make-pose "map" '(0.0 1.0 0.0) '(0.0 0.0 (* pi 0.0)))))
    (perform (an action
                 (type going)
                 (target (a location 
                            (pose ?navigation-goal)))))))

The robot will operate on the table to the right. Here is his base position.

In [ ]:
(defparameter *base-pose-near-table*
  (make-pose "map" '(-2.2d0 -0.20d0 0.0d0) '(0.0d0 0.0d0 (* pi -0.5))))

In [ ]:
(urdf-proj:with-simulated-robot
  (let ((?navigation-goal *base-pose-near-table*))
     (perform (an action
                  (type going)
                  (target (a location 
                             (pose ?navigation-goal)))))))

Besides moving, the robots motors can be also moved. To move the torso up and down, we can use a motion instead of an action. Motions are atomic actions wihtout much logic. They are the most low-level plans, communicating directly with the process-modules.

In [ ]:
(urdf-proj:with-simulated-robot
 (perform (a motion
             (type moving-torso)
             (joint-angle 0.3))))

Also the arms can be moved into standard positions, e.g. to move safely or to kep them out of the cameras field of view.

In [ ]:
(urdf-proj:with-simulated-robot
 (perform (an action
              (type positioning-arm)
              (left-configuration tucked)
              (right-configuration tucked))))

In [ ]:
(urdf-proj:with-simulated-robot
 (perform (an action
              (type positioning-arm)
              (left-configuration park)
              (right-configuration park))))

## Perceiving Objects
Perceiving is also done via action designators. The action of type `detecting` is the one from lesson 2. Let's get out bottle back first.

In [ ]:
(btr-utils:spawn-object 'bottle-1 :bottle :pose (make-pose "map" '(-2.323 -1 0.82) '(0 0 0)))

To perceive something, the robot need to look at the point of interest

In [ ]:
(urdf-proj:with-simulated-robot
  (let ((?navigation-goal *base-pose-near-table*))
    (cpl:par
      ;; Moving the robot near the table.
      (perform (an action
                   (type going)
                   (target (a location
                              (pose ?navigation-goal)))))
      ;; Increasing the height of the torso by setting the joint angle to 0.3 meters
      (perform (a motion
                  (type moving-torso)
                  (joint-angle 0.3)))
      (perform (an action
                  (type positioning-arm)
                  (left-configuration park)
                  (right-configuration park))))))

In [ ]:
(defparameter *downward-look-coordinate*
 (make-pose "base_footprint" '((0.65335d0 0.076d0 0.758d0) (0 0 0 1))))
;; This coordinate frame has base_footprint as reference, which is the reference
;; frame of PR2's base.

In [ ]:
(urdf-proj:with-simulated-robot
  (let ((?looking-direction *downward-look-coordinate*))
    (perform (an action
                 (type looking)
                 (target (a location 
                            (pose ?looking-direction)))))))

In [ ]:
(urdf-proj:with-simulated-robot
  (perform (an action
               (type detecting)
               (object (an object
                          (type bottle))))))

In [ ]:
(defparameter *perceived-bottle* nil)
(urdf-proj:with-simulated-robot
    (setf *perceived-bottle* 
          (perform (an action
                       (type detecting)
                       (object (an object
                                   (type bottle)))))))

## Picking up objects

In [ ]:
(urdf-proj:with-simulated-robot 
  (let ((?perceived-bottle *perceived-bottle*))
   (perform (an action
                (type picking-up)
                 (arm (right))
                 (grasp right-side)
                 (object ?perceived-bottle)))
  ;; Parking the right arm after grasping, bringing the bottle close to the robot
  ;; and freeing up the field of view.
    (exe:perform
           (desig:an action
                     (type parking-arms)
                     (arms (right))))))

## Placing Objects

In [ ]:
(defparameter *final-object-destination*
  (make-pose "map" '((-0.8 2 0.9) (0 0 0 1))))
 
(defparameter *base-pose-near-counter*
  (make-pose "map" '((-0.150d0 2.0d0 0.0d0) (0.0d0 0.0d0 -1.0d0 0.0d0))))

In [ ]:
(roslisp-utilities:startup-ros)

In [ ]:
(demo::initialize)

(spawn-object '((-2.323 -1 0.82) (0 0 0 1)))

(urdf-proj:with-simulated-robot
 (let ((?navigation-goal 
        (cl-transforms-stamped:make-pose-stamped 
            "map" 0.0 
         (cl-transforms:make-3d-vector -2.2d0 -0.20d0 0.0d0)
         (cl-transforms:euler->quaternion :az (* pi -0.50)))))
      (perform (an action
                 (type going)
                 (target (a location 
                            (pose ?navigation-goal))))))
 

  (let ((?looking-direction (make-pose "base_footprint" '((0.65335d0 0.076d0 0.758d0) (0 0 0 1)))))
    (perform (an action
                 (type looking)
                 (target (a location 
                            (pose ?looking-direction))))))
 
 (let ((?perceived-bottle 
        (perform (an action
                     (type detecting)
                     (object (an object
                                 (type bottle))))))
       (?nav-goal (make-pose "map" '((-0.15d0 2.0d0 0.0d0) (0.0d0 0.0d0 -1.0d0 0.0d0))))
       (?drop-pose (make-pose "map" '((-0.8 2 0.9) (0 0 0 1)))))
  
  (perform (an action
                (type picking-up)
                 (arm (right))
                 ;(side right-side)
                 (object ?perceived-bottle)))
  ;; Parking the right arm after grasping, bringing the bottle close to the robot
  ;; and freeing up the field of view.
    (perform (an action
                 (type parking-arms)))
   (perform (a motion 
               (type moving-torso)
               (joint-angle 0.3)))
   (perform (an action
                (type going)
                (target (a location
                           (pose ?nav-goal)))))
 
   (perform (an action
                (type placing)
                (arm (right))
                (object ?perceived-bottle)
                (target (a location
                           (pose ?drop-pose)))))))

In [ ]:


(defun move-kitchen-joint (&key (joint-name "iai_fridge_door_joint")
                             (joint-angle 0.2d0))
  (btr:set-robot-state-from-joints
   `((,joint-name  ,joint-angle))
   (btr:get-environment-object)))

(defun add-objects-to-mesh-list (&optional (ros-package "cram_bullet_world_tutorial"))
  (mapcar (lambda (object-filename-and-object-extension)
            (declare (type list object-filename-and-object-extension))
            (destructuring-bind (object-filename object-extension)
                object-filename-and-object-extension
              (let ((lisp-name (roslisp-utilities:lispify-ros-underscore-name
                                object-filename :keyword)))
                (pushnew (list lisp-name
                               (format nil "package://~a/resource/~a.~a"
                                       ros-package object-filename object-extension)
                               nil)
                         btr::*mesh-files*
                         :key #'car)
                lisp-name)))
          (mapcar (lambda (pathname)
                    (list (pathname-name pathname) (pathname-type pathname)))
                  (directory (physics-utils:parse-uri
                              (format nil "package://~a/resource/*.*" ros-package))))))
                              

;;;; **** ARM Parking
(defun park-arms ()
  (pp-plans::park-arms))

(defun park-arm (arm)
  (pp-plans::park-arms :arm arm))

;;**** Failure Handler
(defmacro handle-failure (errors program-body &body error-handler-body)
  `(progn
    (cpl:with-failure-handling
        ((,errors (e)
           (print e)
           ,@error-handler-body))
    ,@program-body)))


(defparameter *pose-bottle-1*
  (cl-transforms-stamped:make-pose-stamped
   "map" 0.0
   (cl-transforms:make-3d-vector -2 -0.9d0 0.86d0)
   (cl-transforms:make-identity-rotation)))

(defparameter *pose-bottle-2*
  (cl-transforms-stamped:make-pose-stamped
   "map" 0.0
   (cl-transforms:make-3d-vector -0.8 2 0.9)
   (cl-transforms:make-identity-rotation)))

(defparameter *pose-meal-table*
  (cl-transforms-stamped:make-pose-stamped
   "map" 0.0
   (cl-transforms:make-3d-vector -0.15 2.0 0)
   (cl-transforms:make-quaternion 0.0d0 0.0d0 -1.0d0 0.0d0)))

(defparameter *pose-counter*
  (cl-transforms-stamped:make-pose-stamped
   "map" 0.0
   (cl-transforms:make-3d-vector -1.8547d0 -0.381d0 0.0d0)
   (cl-transforms:axis-angle->quaternion (cl-transforms:make-3d-vector 0 0 1) (/ pi -2))))

(defun spawn-two-bottles ()
  (unless (assoc :bottle btr::*mesh-files*)
    (add-objects-to-mesh-list))
  (prolog:prolog
   `(and (btr:bullet-world ?world)
         (assert (btr:object ?world :mesh bottle-1 ((-2 -0.9 0.860) (0 0 0 1))
                             :mass 0.2 :color (1 0 0) :mesh :bottle))
         (assert (btr:object ?world :mesh bottle-2 ((-0.8 2 0.9) (0 0 0 1))
                             :mass 0.2 :color (0 1 0) :mesh :bottle))
         (btr:simulate ?world 100))))

(defun navigate-to (?navigation-goal)
  (exe:perform (desig:a motion
                        (type going)
                        (pose ?navigation-goal))))

(defun look-at (?point-of-interest)
  (exe:perform (desig:a motion
                        (type looking)
                        (pose ?point-of-interest))))

(defun get-perceived-bottle-desig ()
  (let* ((?bottle-desig (desig:an object (type bottle)))
         (?perceived-bottle-desig (exe:perform
                                   (desig:a motion
                                            (type detecting)
                                            (object ?bottle-desig)))))
    ?perceived-bottle-desig))

(defun pick-up (?object-designator &optional (?arm :right))
  (exe:perform (desig:an action
                         (type picking-up)
                         (arm ?arm)
                         (object ?object-designator))))

(defun place-down (?pose ?object ?arm)
  (exe:perform (desig:an action
                         (type placing)
                         (arm ?arm)
                         (object ?object)
                         (target (desig:a location (pose ?pose))))))

(defun test-switch-two-bottles ()
  (spawn-two-bottles)
  (proj:with-projection-environment urdf-proj:urdf-bullet-projection-environment
    (cpl:top-level
      ;; Go to counter top and perceive bottle
      (let ((?navigation-goal *pose-counter*)
            (?ptu-goal 
              (cl-transforms-stamped:make-pose-stamped
               "base_footprint"
               0.0
               (cl-transforms:make-3d-vector 0.65335d0 0.076d0 0.758d0)
               (cl-transforms:make-identity-rotation))))
        (cpl:par
          ;; Move torso up
          (exe:perform
           (desig:a motion (type moving-torso) (joint-angle 0.3)))
          (exe:perform
           (desig:an action
                     (type parking-arms)))
          (navigate-to ?navigation-goal))
        (look-at ?ptu-goal))
      ;; Pick up bottle-1 with right arm.
      (let ((?perceived-bottle-1 (get-perceived-bottle-desig)))
        (pick-up ?perceived-bottle-1 :right)
        (exe:perform
         (desig:an action
                   (type parking-arms)
                   (arms (:right))))
        ;; Move to the meal table
        (let ((?pose *pose-meal-table*))
          (navigate-to ?pose))
        ;; Pick up bottle-2 with left arm
        (let ((?perceived-bottle-2 (get-perceived-bottle-desig)))
          (pick-up ?perceived-bottle-2 :left)
          ;; Move left arm out of sight
          (exe:perform
           (desig:an action
                     (type parking-arms)
                     (arms (:left))))
          ;; Place bottle-1 on second table
          (let ((?drop-pose *pose-bottle-2*))
            (place-down ?drop-pose ?perceived-bottle-1 :right))
          ;; Move right arm out of sight
          (exe:perform
           (desig:an action
                     (type parking-arms)
                     (arms (:right))))
          ;; Move to the counter table
          (let ((?navigation-goal *pose-counter*))
            (navigate-to ?navigation-goal))
          ;; Place bottle-2 on the counter
          (let ((?drop-pose *pose-bottle-1*))
            (place-down ?drop-pose ?perceived-bottle-2 :left))
          (exe:perform
           (desig:an action
                     (type parking-arms))))))))